# House Price Predictor

* In this notebook I will be uisng machine learning algorithms to determine the prices of houses using the data set from Kaggle.

### Import Requried Libraires for analysis

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

### Import the datasets

In [3]:
# Import the training data
path = Path("Data/train.csv")
test_path = Path("Data/test.csv")

In [4]:
# create dataframe from train csv
train_df = pd.read_csv(path)
train_df.head(
)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# create dataframe from test csv
test_df = pd.read_csv(test_path)
test_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [6]:
# check for null values in both datasets
test_df.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [7]:
train_df.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [8]:
# Check columns types and correct if necessary
# Call the object columns to make sure they should be categorized as object or if shouls be replaced
train_df.dtypes[(train_df.dtypes!="int64") & (train_df.dtypes!="float64")]

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
GarageCond       object
PavedDrive       object
PoolQC           object
Fence            object
MiscFeature      object
SaleType        

In [9]:
test_df.dtypes[(test_df.dtypes!="int64") & (test_df.dtypes!="float64")]

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
GarageCond       object
PavedDrive       object
PoolQC           object
Fence            object
MiscFeature      object
SaleType        

In [10]:
# Use this helper function to assit in dropping null values and use the mean ti fill the gap
def fill_in(data):
    for i in data.columns:
        # if data type is equal to object we leave it alone
        if data[i].dtype == "object":
            data[i] = data[i].fillna("")
        # if data type is a intger or float we want to take the mean of that and fill it in for the null values
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data

In [11]:
# pass the train df into thi function
train_df = fill_in(train_df)

In [12]:
# pass the test df into this function
test_df = fill_in(test_df)

## Prepare the datasets to but used in the machine learning algorithm

In [13]:
# Convert all the data into numerical data
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

In [14]:
train_df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [15]:
# check the target column to determine if it is a classification or regression model
# sale price is numerical so it is going to be a regression model
train_df["SalePrice"]

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [16]:
# reshape the target model so the shape has a 1 to match with the training (X) set
y = train_df["SalePrice"].values.reshape(-1, 1)

In [17]:
# store the training data into the X variable
# drop the target column when making the X or the training set
X = train_df.drop(columns = ["SalePrice"])
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [18]:
# make sure the y shape has a 1 in it
# reshape the model so it is (1460, 1)
y.shape

(1460, 1)

In [19]:
X.shape

(1460, 305)

### Fit the data to a Scaler

In [20]:
# create a scaler for the model
scaler = StandardScaler()

# scale the training data

X_scaler = scaler.fit(X)

# use the X_scaler to scale the test data
X_train_scaled = X_scaler.transform(X)
#X_test_scaled = X_scaler.transform(X_test)

### Incorporate PCA to assist on the speed of the algorithm since there is a lot of columns

In [21]:
# incorporate PCA into the train model
pca = PCA(n_components = 3)

X_pca_scaled = pca.fit_transform(X)
#test_pca_scaled = pca.fit_transform(X_test_scaled)

In [22]:
# split the test csv into X_test
X_test = test_df
# transform the test dataframe so it incorporates PCA 
X_test_pca_scaled = pca.fit_transform(X_test)

## Import Regression Model (Gradient Boosting)

In [23]:
# use gradient boosting regessor
from sklearn.ensemble import GradientBoostingRegressor
# import the relevent metrics to determine how our model did mse, r2, score
from sklearn.metrics import mean_squared_error, r2_score


In [24]:
gbr = GradientBoostingRegressor()

In [25]:
# Use hyperparameter tunning to assist in getting the best parameters for the model

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
## the more parameters that are supplied the longer the model will take to fit

In [28]:
grid_search = {"n_estimators":[50, 75, 100],
              "learning_rate":[0.01, 0.5, 1],
              "max_depth":[2, 3, 4],
              "min_samples_split": [2, 3, 4],
              "min_samples_leaf": [2, 3, 4]
              }

In [29]:
# Pass the grid_search varaible into GridSearchCV
param_search = GridSearchCV(estimator = gbr, param_grid = grid_search, scoring = "neg_mean_squared_error")

In [30]:
param_search.fit(X_pca_scaled, y)

GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.5, 1],
                         'max_depth': [2, 3, 4], 'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [50, 75, 100]},
             scoring='neg_mean_squared_error')

In [31]:
# Obatin the best parameters by using bese_params_
param_search.best_params_

{'learning_rate': 0.5,
 'max_depth': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 50}

In [32]:
# Create model based on above parameters
gradient_model = GradientBoostingRegressor(n_estimators = 50, 
                                           learning_rate = 0.5, 
                                           max_depth = 4, 
                                           min_samples_leaf = 2, 
                                           min_samples_split = 4, 
                                           random_state = 78)

In [33]:
# fit the X_pca_scaled and y to the gradient_model
gradient_model.fit(X_pca_scaled, y)

GradientBoostingRegressor(learning_rate=0.5, max_depth=4, min_samples_leaf=2,
                          min_samples_split=4, n_estimators=50,
                          random_state=78)

In [34]:
# predict the X_pca_scaled
predictions_x_train = gradient_model.predict(X_pca_scaled)
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]], dtype=int64)

In [35]:
# determine the accuracy of the model by using the r2 scoring method
# A score of 1 means the model preformed perfectly while a score of 0 means it scored poorly
# in this case we have a score of .93 which means it performed well on unseen data
r2_score(predictions_x_train, y)

0.9365254484710016

In [36]:
# Calculate the mean_squared_error on the predictions_x_train and y
# We then need to take the square root becasue the error were raised to the second power in order to remove negative values
# By taking the square root we need are getting the values back to scale
np.sqrt(mean_squared_error(predictions_x_train, y))

18871.292746371862

# Test set analysis

In [37]:
# precit the model on the X_test_pca_scaled
predictions_x_test = gradient_model.predict(X_test_pca_scaled)

In [47]:
predictions_x_test

array([124352.03827225, 284157.95099538, 153533.44291048, ...,
       170995.28846375, 157085.28160688, 198902.94275737])

In [46]:
pd.DataFrame({'id':X_test['Id'], 'SalePrice':predictions_x_test}).to_csv('Final.csv', index=False)